In [348]:
import pandas as pd
from tqdm import tqdm

# Data preprocess

In [426]:
train_data = pd.read_csv("train.csv", index_col=0)

In [427]:
# delexicalization 
def replace_something(x, subj, obj):
    
    subj_start = eval(x.subject_entity)['start_idx']
    subj_end = eval(x.subject_entity)['end_idx']
    
    obj_start = eval(x.object_entity)['start_idx']
    obj_end = eval(x.object_entity)['end_idx']
    
    if subj_start < obj_start:
        
        new_sentence = x.sentence[:subj_start] + subj + x.sentence[subj_end+1:obj_start] + obj + x.sentence[obj_end + 1:]
        return new_sentence
            
    else:
        new_sentence = x.sentence[:obj_start] + obj + x.sentence[obj_end+1:subj_start] + subj + x.sentence[subj_end + 1:]
        return new_sentence


In [428]:
train_data['new_sentence'] = train_data.apply(lambda x: replace_something(x,"[subj]","[obj]"), axis=1)

In [429]:
train_data['new_sentence'].iloc[0]

'〈Something〉는 [obj]이 쓰고 [subj]가 1969년 앨범 《Abbey Road》에 담은 노래다.'

# Back Translation

In [346]:
from pororo import Pororo
from typing import Optional
from pororo.tasks.machine_translation import PororoTranslationFactory, PororoTransformerTransMulti
from pororo.tasks.utils.download_utils import download_or_load

## Pororo overrdie

In [347]:
class batch_translation(PororoTransformerTransMulti):
    
    def __init__(self,model, config, tokenizer, sent_tokenizer, langtok_style):
        super().__init__(model, config, tokenizer, sent_tokenizer, langtok_style)
        self.model = model
        self._tokenizer = tokenizer
        self._sent_tokenizer = sent_tokenizer
        self._langtok_style = langtok_style
    
    def _preprocess(self, text, src: str, tgt: str):
        """
        Preprocess non-chinese input sentence to replace whitespace token with whitespace

        Args:
            text (str): non-chinese sentence
            src (str): source language
            tgt (str): target language

        Returns:
            str: preprocessed non-chinese sentence

        """
        
        text_series = pd.Series(text)
        
        if src == "en":
            text_series = text_series.apply(lambda x : " ".join(self._tokenizer.segment(x.strip())))
        else:
            text_series = text_series.apply(lambda x : " ".join([c if c != " " else "▁" for c in x.strip()]))
            
        text_series = text_series.apply(lambda x : f"{self._langtok(src, self._langtok_style)} {x} {self._langtok(tgt, self._langtok_style)}")
        return text_series

    def _postprocess(self, output):
        """
        Postprocess output sentence to replace whitespace

        Args:
            output (str): output sentence generated by model

        Returns:
            str: postprocessed output sentence

        """
        output = pd.Series(output)
        output = output.apply(lambda x : x.replace(" ", "").replace("▁", " ").strip())
        return output
    
    def predict(
        self,
        text,
        src: str,
        tgt: str,
        beam: int = 5,
        temperature: float = 1.0,
        top_k: int = -1,
        top_p: float = -1,
        no_repeat_ngram_size: int = 4,
        len_penalty: float = 1.0,
        **kwargs):
        
        text = self._preprocess(text, src, tgt)

        sampling = False

        if top_k != -1 or top_p != -1:
            sampling = True

        output = self._model.translate(
            text,
            beam=beam,
            sampling=sampling,
            temperature=temperature,
            sampling_topk=top_k,
            sampling_topp=top_p,
            max_len_a=1,
            max_len_b=50,
            no_repeat_ngram_size=no_repeat_ngram_size,
            lenpen=len_penalty,
        )
        
        output = self._postprocess(output)
        return output
        
        # return text
        
class batch_TranslationFactory(PororoTranslationFactory):
    
    def __init__(
        self,
        task: str,
        lang: str,
        model: Optional[str],
        tgt: str = None):
        
        super().__init__(task, lang, model)
        self._src = self.config.lang
        self._tgt = tgt
        
    def load(self, device: str):
        """
        Load user-selected task-specific model

        Args:
            device (str): device information

        Returns:
            object: User-selected task-specific model

        """
        from pororo.tasks import PororoTokenizationFactory

        sent_tokenizer = (lambda text, lang: PororoTokenizationFactory(
            task="tokenization",
            lang=lang,
            model=f"sent_{lang}",
        ).load(device).predict(text))

        if "multi" in self.config.n_model:
            from fairseq.models.transformer import TransformerModel

            from pororo.tasks.utils.tokenizer import CustomTokenizer

            load_dict = download_or_load(
                f"transformer/{self.config.n_model}",
                self.config.lang,
            )

            model = (TransformerModel.from_pretrained(
                model_name_or_path=load_dict.path,
                checkpoint_file=f"{self.config.n_model}.pt",
                data_name_or_path=load_dict.dict_path,
                source_lang=load_dict.src_dict,
                target_lang=load_dict.tgt_dict,
            ).eval().to(device))

            tokenizer = CustomTokenizer.from_file(
                vocab_filename=f"{load_dict.src_tok}/vocab.json",
                merges_filename=f"{load_dict.src_tok}/merges.txt",
            )

            if "mtpg" in self.config.n_model:
                langtok_style = "mbart"
            elif "m2m" in self.config.n_model:
                langtok_style = "multilingual"
            else:
                langtok_style = "basic"

            return batch_translation(
                model,
                self.config,
                tokenizer,
                sent_tokenizer,
                langtok_style,
            )

## translation

In [193]:
mt = batch_TranslationFactory(task="translation", lang="multi", model='transformer.large.multi.mtpg')
translator= mt.load('cuda')

In [430]:
#train_data = pd.read_csv('train.csv',index_col=0).iloc[:50]
sentence = train_data.new_sentence.iloc[:50]

In [431]:
def back_translation(translator, sentences, src, tgt, bs):
    
    translated_sentence = pd.Series()
    for i in tqdm(range(1,(len(sentences) // bs) + 2)):

        batch = sentences.iloc[bs*(i-1):bs*i]

        ko_to_en = translator.predict(batch,src=src ,tgt=tgt)
        # back translation
        en_to_ko = translator.predict(ko_to_en,src=tgt,tgt=src)

        translated_sentence = translated_sentence.append(en_to_ko)
        
    translated_sentence = translated_sentence.reset_index(drop=True)
        
    return translated_sentence

In [432]:
result = back_translation(translator, sentence, src='ko',tgt='ja',bs=32)

<ipython-input-431-fc4bfbba67e9>:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  translated_sentence = pd.Series()
100%|██████████| 2/2 [00:19<00:00,  9.61s/it]


# Replace Data

In [478]:
target = train_data.iloc[:50]
target['back_translated'] = result

<ipython-input-478-b400957a1cfb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target['back_translated'] = result


In [479]:
def new_function(x, subj_marker, obj_marker):
    
    subj = eval(x.subject_entity)['word']
    obj = eval(x.object_entity)['word']
    
    if subj_marker in x.back_translated and obj_marker in x.back_translated:
        new_sentence = x.back_translated.replace(subj_marker,subj)
        new_sentence = new_sentence.replace(obj_marker,obj)
        return new_sentence
    else:
        return None

In [480]:
target['back_translated'] = target.apply(lambda x : new_function(x,"[subj]","[obj]"), axis=1)

『소메팅』은 『오브제이』가 써서 『수브제이』가 1969년 앨범 『아베이로드』에 담긴 노래다.
호남 기반의 발른 미래당·[obj]·[수부지]가 우여곡절 끝에 맞춰 민생당(가칭)으로 재생한다.
K리그2에서 성적 1위를 달리고 있는 [수브지]는 26일[obj]에서 관중 유치의 성과와 마케팅의 성과를 인정받아 ‘풀스타디움상’과 ‘플래스타디움상’을 수상했다.
균일가의 생활용품점(주)[subj](대표[obj]은 코로나 19바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 신고했다고 합니다.
[obj]년 프로야구 드래프트 1위로 [subj]에 입단하여 등기번호는 8번으로 배정되었다.
유엔, 유럽의회, [수브지](obj), 국제이민기관, 세계보건기관(WHO), 지중해연합, 이슬람협력기관, 유럽의 안전협력기관, 국제통화기금, 세계무역기관, 프랑코포니.
그에 따라 나폴리와 계약을 연장한 마라도나는 [obj]년 팀을 UEFA컵 정상으로 이끌었고, 다음 해에는 유럽 챔피언[subj]을 상대방에게 승리를 거두어 다시 한 번 세리에A에서 정상에 등극했다.
[Subj](Pakyong-Hom, [obj](음력 3월 19일)(음력 3월 19일)~2009년 11월 4일)는 서울에서 태어난 대한민국 기업인으로서 두산그룹 회장, KBO 총재 등을 지냈다.
중공군에 완전히 대항할 수 없을 정도로 약해진 국민당은 [obj]로 수도를 옮기기로 결정하고, 남아있는 [subj]군의 병력과 국가, 개인의 재산 등을 계속해서 대만으로 옮기기 시작했고, 12월에는 중앙정부 기구도 모두 이전하여 태북시를 중화민국의 새로운 수도로 삼았다.
특히 김둔영 원수의 경제부총리, 김두건 국회의원, [수브지] 국회의원, 김정민 국회의원, 오제세 국회의원, 체운열 국회의원, 김정우 국회의원, 권7승 국회의원, 맹성규 국회의원 등 [obj] 국회의원 8명이 비디오 축하 메시지를 보내 눈길을 끌었다.
[Subj]는 [obj] 클럽 타일리스의 유소년 팀에서 축구를 시작했다.
미국[obj][subj](본명 빌리오 코너, 19)가

<ipython-input-480-3d049ed317fa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target['back_translated'] = target.apply(lambda x : new_function(x,"[subj]","[obj]"), axis=1)


In [488]:
target

,guid,sentence,subject_entity,object_entity,label,source,new_sentence,back_translated
0,klue-re-v1_train_00000,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",0,wikipedia,〈Something〉는 [obj]이 쓰고 [subj]가 1969년 앨범 《Abbey...,None
1,klue-re-v1_train_00001,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",0,wikitree,호남이 기반인 바른미래당·[obj]·[subj]이 우여곡절 끝에 합당해 민생당(가칭...,None
2,klue-re-v1_train_00002,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",5,wikitree,K리그2에서 성적 1위를 달리고 있는 [subj]는 지난 26일 [obj]으로부터 ...,None
3,klue-re-v1_train_00003,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",10,wikitree,균일가 생활용품점 (주)[subj](대표 [obj])는 코로나19 바이러스로 어려움...,균일가의 생활용품점(주)아성다이소(대표박정부은 코로나 19바이러스로 어려움을 겪고 ...
4,klue-re-v1_train_00004,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",0,wikipedia,[obj]년 프로 야구 드래프트 1순위로 [subj]에게 입단하면서 등번호는 8번으...,1967년 프로야구 드래프트 1위로 요미우리 자이언츠에 입단하여 등기번호는 8번으로...
5,klue-re-v1_train_00005,": 유엔, 유럽 의회, 북대서양 조약 기구 (NATO), 국제이주기구, 세계 보건 ...","{'word': '북대서양 조약 기구', 'start_idx': 13, 'end_i...","{'word': 'NATO', 'start_idx': 25, 'end_idx': 2...",4,wikipedia,": 유엔, 유럽 의회, [subj] ([obj]), 국제이주기구, 세계 보건 기구 ...",None
6,klue-re-v1_train_00006,그에 따라 나폴리와 계약을 연장한 마라도나는 1989년 팀을 UEFA컵 정상으로 인...,"{'word': 'AC 밀란', 'start_idx': 64, 'end_idx': ...","{'word': '1989', 'start_idx': 25, 'end_idx': 2...",0,wikipedia,그에 따라 나폴리와 계약을 연장한 마라도나는 [obj]년 팀을 UEFA컵 정상으로 ...,그에 따라 나폴리와 계약을 연장한 마라도나는 1989년 팀을 UEFA컵 정상으로 이...
7,klue-re-v1_train_00007,"박용오(朴容旿, 1937년 4월 29일(음력 3월 19일)(음력 3월 19일) ~ ...","{'word': '박용오', 'start_idx': 0, 'end_idx': 2, ...","{'word': '1937년 4월 29일', 'start_idx': 9, 'end_...",12,wikipedia,"[subj](朴容旿, [obj](음력 3월 19일)(음력 3월 19일) ~ 2009...",None
8,klue-re-v1_train_00008,중공군에게 온전히 대항할 수 없을 정도로 약해진 국민당은 타이베이로 수도를 옮기는 ...,"{'word': '중화민국', 'start_idx': 59, 'end_idx': 6...","{'word': '타이베이', 'start_idx': 32, 'end_idx': 3...",3,wikipedia,중공군에게 온전히 대항할 수 없을 정도로 약해진 국민당은 [obj]로 수도를 옮기는...,중공군에 완전히 대항할 수 없을 정도로 약해진 국민당은 타이베이로 수도를 옮기기로 ...
9,klue-re-v1_train_00009,"특히 김동연 전 경제부총리를 비롯한 김두관 국회의원, 안규백 국회의원, 김종민 국회...","{'word': '안규백', 'start_idx': 30, 'end_idx': 32...","{'word': '더불어민주당', 'start_idx': 100, 'end_idx'...",18,wikitree,"특히 김동연 전 경제부총리를 비롯한 김두관 국회의원, [subj] 국회의원, 김종민...",None


In [491]:
target.iloc[12].sentence

'2009년 9월, 미국 프로 야구 필라델피아 필리스 소속의 야구 선수 박찬호는 《MBC 스페셜-박찬호는 당신을 잊지 않았다》 편에서 “최진실 씨의 아픔과 죽음의 고통을 이해합니다. 최진실 씨 사건에 눈물을 흘렸습니다. 저도 죽으려고 마음을 먹었던 적이 있었습니다. 잘하려고 애를 쓰는데 비난과 비판이 쏟아졌습니다. 머리가 빠지고 너무 힘들었습니다”라고 말하며 최진실의 죽음에 대해 안타까움을 표현했다.'

In [492]:
target.iloc[12].back_translated

'2009년 9월, 미국프로야구필라델피아 필리스 소속의 야구선수박찬호는 "MBC 스페셜 박찬호는 당신을 잊지 않았다"는 분으로, "체진실 씨의 고통과 죽음의 고통을 이해하고 있습니다. 체진실 씨의 사건에 눈물을 흘렸습니다. 저도 죽은 적이 있었습니다. 잘하려고 노력하는데 비난과 비판이 쏟아졌습니다. 머리가 빠져서 매우 힘들었습니다"라고 말하며 최진실의 죽음에 대해 유감을 표했습니다.'